# Clasificador Labubu vs Lafufu con PyTorch

Este notebook entrena un modelo de deep learning para distinguir entre imágenes de Labubu y Lafufu usando PyTorch. El entrenamiento se realizó originalmente en Google Colab. Se puede ver el notebook original aquí:

[Google Colab - Labubu vs Lafufu](https://colab.research.google.com/drive/1fOcjyK30MnSaNEbQImoCwwtaMCU4_wYz?usp=sharing)

Adicionalmente, puedes probar el modelo aquí:

[Modelo - Labubu vs Lafufu](https://huggingface.co/spaces/sininter/Labubu-VS-Lafufu)

## Paso 1. Importar librerías y configuración
Se importan las librerías necesarias para el manejo de datos, imágenes, visualización y deep learning. También se configuran los warnings para evitar mensajes innecesarios.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import kagglehub

## Paso 2. Descargar y preparar el dataset
Se descarga el dataset de Kaggle y se muestra la ruta donde se almacena para su posterior uso.

In [ ]:
path = kagglehub.dataset_download("sebaalmokdad/labubu-vs-lafufu")
print("Path to dataset:" , path)
data_dir = path
print(data_dir)

## Paso 3. Definir la arquitectura de la red
Se define una red neuronal convolucional simple en PyTorch para clasificar las imágenes en dos clases.

In [ ]:
class CNNWithPyTorch(nn.Module):
    def __init__(self):
        super(CNNWithPyTorch, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, 2)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 56 * 56)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## Paso 4. Transformaciones y carga de datos
Se aplican transformaciones a las imágenes y se cargan en DataLoaders para entrenamiento, validación y prueba.

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

dataset = datasets.ImageFolder(root=data_dir, transform=transform)
classes = dataset.classes
print("✅ Classes:", classes)

train = datasets.ImageFolder(root=data_dir, transform=transform)
train_loader = DataLoader(train, batch_size=32, shuffle=True)
print("✅ Classes found:", train.classes)
print("✅ Class to index mapping:", train.class_to_idx)
print("✅ Total samples:", len(train))

train_size = int(0.7 * len(train))
val_size = int(0.2 * len(train))
test_size = len(train) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(train, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

## Paso 5. Visualización de imágenes de ejemplo
Se muestran algunas imágenes del dataset junto a sus etiquetas para verificar la carga correcta.

In [ ]:
def view_images(img, title):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.title(title)
    plt.axis('off')
    plt.show()

dataiter = iter(train_loader)
images, labels = next(dataiter)
for i in range(4):
    view_images(images[i], title=f"Label: {labels[i].item()} ({train.classes[labels[i]]})")

## Paso 6. Entrenamiento y validación del modelo
Se entrena la red neuronal y se evalúa su desempeño en el conjunto de validación durante varias épocas.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNNWithPyTorch().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 15
train_losses, val_losses, train_accs, val_accs = [], [], [], []

for epoch in range(epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    val_loss /= len(val_loader)
    val_acc = 100 * val_correct / val_total
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    print(f"Epoch {epoch+1}/{epochs} Train Loss={train_losses[-1]:.4f}, Train Acc={train_accs[-1]:.2f}% Val Loss={val_losses[-1]:.4f}, Val Acc={val_accs[-1]:.2f}%")

## Paso 7. Visualización de curvas de aprendizaje
Se grafican las curvas de pérdida y precisión para observar el comportamiento del modelo durante el entrenamiento.

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Val Loss")
plt.legend(); plt.title("Loss Curve")
plt.subplot(1,2,2)
plt.plot(train_accs, label="Train Acc")
plt.plot(val_accs, label="Val Acc")
plt.legend(); plt.title("Accuracy Curve")
plt.show()

## Paso 8. Matriz y métricas
Se calcula la matriz y el reporte de clasificación para evaluar el desempeño del modelo.

In [ ]:
classes = dataset.classes
all_preds, all_labels = [], []
model.eval()
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
disp.plot(cmap="Blues")
plt.title("Confusion Matrix")
plt.show()
print(classification_report(all_labels, all_preds, target_names=classes))

## Paso 9. Predicción con imágenes nuevas
Se define una función para predecir la clase de una imagen nueva usando el modelo entrenado.

In [ ]:
def predict_image(img_path):
    img = Image.open(img_path).convert("RGB")
    transform_test = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
    ])
    img = transform_test(img).unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        outputs = model(img)
        _, pred = torch.max(outputs, 1)
    return classes[pred.item()]

print("Prediction:", predict_image("/root/.cache/kagglehub/datasets/sebaalmokdad/labubu-vs-lafufu/versions/1/labubu_images/104.jpg"))

## Paso 10. Interfaz interactiva con Gradio
Se crea una interfaz web sencilla para cargar imágenes y obtener la predicción del modelo junto a una gráfica de probabilidades.

In [ ]:
!pip install torch torchvision pillow gradio
import gradio as gr

title = "Clasificador Labubu vs. Lafufu (PyTorch) con Gráfica"
description = "Sube una imagen para obtener la predicción y ver la probabilidad de cada clase."
salida_prediccion = gr.Label(num_top_classes=2, label="Probabilidades")
salida_grafica = gr.Plot(label="Visualización de la Predicción")

def classify_labufu_with_plot(img):
    pred = predict_image(img)
    # Aquí deberías agregar la lógica para devolver también la gráfica de probabilidades
    # Por simplicidad, solo devolvemos la predicción y una figura vacía
    fig, ax = plt.subplots()
    ax.set_title("Probabilidades")
    return {"label": pred, "confidences": []}, fig

interfaz_plot = gr.Interface(
    fn=classify_labufu_with_plot,
    inputs=gr.Image(type="pil", label="Sube una imagen (224x224)"),
    outputs=[salida_prediccion, salida_grafica],
    title=title,
    description=description,
    theme=gr.themes.Soft()
)

interfaz_plot.launch(share=True)

**Créditos:** Gran parte de este código está basado en el trabajo de [yashdev01 en Kaggle](https://www.kaggle.com/code/yashdev01/labubu-vs-lafufu-classification) y el dataset de [sebaalmokdad](https://www.kaggle.com/sebaalmokdad).